In [1]:
import os
import argparse
import torch
import soundfile as sf
import logging
from datetime import datetime
import platform
import sys
import random
sys.path.append(os.path.abspath("./cli"))
sys.path.append(os.path.abspath("./sparktts"))
from sparktts.utils.token_parser import LEVELS_MAP,LEVELS_MAP_UI, GENDER_MAP, TASK_TOKEN_MAP


from SparkTTS import SparkTTS


/home/qihan/miniconda3/envs/sparktts/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


def initialize_model(model_dir="pretrained_models/Spark-TTS-0.5B", device=0):
    """Load the model once at the beginning."""

    # Determine appropriate device based on platform and availability
    if platform.system() == "Darwin":
        # macOS with MPS support (Apple Silicon)
        device = torch.device(f"mps:{device}")
    elif torch.cuda.is_available():
        # System with CUDA support
        device = torch.device(f"cuda:{device}")
    else:
        # Fall back to CPU
        device = torch.device("cpu")

    model = SparkTTS(model_dir, device)
    return model


def run_tts(
    text,
    model,
    prompt_text=None,
    prompt_speech=None,
    gender=None,
    pitch=None,
    speed=None,
    save_dir="example/results",
):
    """Perform TTS inference and save the generated audio."""

    if prompt_text is not None:
        prompt_text = None if len(prompt_text) <= 1 else prompt_text

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Generate unique filename using timestamp
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    save_path = os.path.join(save_dir, f"{timestamp}.wav")


    # Perform inference and save the output audio
    with torch.no_grad():
        wav = model.inference(
            text,
            prompt_speech,
            prompt_text,
            gender,
            pitch,
            speed,
        )

        sf.write(save_path, wav, samplerate=16000)

    return save_path



In [3]:
input_text="Happy Birthday Jeff!"

In [4]:
model=initialize_model()
save_dir=f"example/results/{input_text.replace(' ', '_')}"
os.makedirs(save_dir, exist_ok=True)



/home/qihan/miniconda3/envs/sparktts/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb


In [35]:
#N= number of sound source


N=5
random_list_pitch = [random.randint(4, 4) for _ in range(N)]
random_list_speed = [random.randint(1,5) for _ in range(N)]
random_list_gender = [random.randint(1, 2) for _ in range(N)]




GENDER_MAP_UI={1:"female",2:"male"}
for i in range(N):

    run_tts(
        input_text,
        model,
        save_dir=save_dir,
        gender=GENDER_MAP_UI[random_list_gender[i]],
        pitch=LEVELS_MAP_UI[random_list_pitch[i]],
        speed=LEVELS_MAP_UI[random_list_speed[i]],
    )



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
#Apply additional variation to the sound


In [43]:
import os
from pydub import AudioSegment

# Function to normalize audio
def normalize_audio(audio, target_dBFS=-20.0):
    change_in_dBFS = target_dBFS - audio.dBFS
    return audio.apply_gain(change_in_dBFS)

# Normalize the loudness list to a range of -10 to 10 dB
def normalize_loudness(loudness_list):
    max_value = max(abs(x) for x in loudness_list)
    if max_value == 0:
        return [0] * len(loudness_list)  # All zeros if max_value is zero
    return [int((x / max_value) * 10) for x in loudness_list]

# Directory containing the WAV files
files = [f for f in os.listdir(save_dir) if f.endswith('.wav')]
loudness_list = [random.randint(1, 100) for _ in range(len(files))]
# Example loudness list (replace with your own values)

if len(loudness_list) != len(files):
    raise ValueError("Length of loudness list must match the number of audio files.")

# Normalize the loudness list
normalized_loudness_list = normalize_loudness(loudness_list)
print(f"Normalized loudness list: {normalized_loudness_list}")

# Determine the maximum duration among all audio files
max_duration = 0
audio_segments = []

for file in files:
    file_path = os.path.join(save_dir, file)
    audio = AudioSegment.from_wav(file_path)
    audio_segments.append(audio)
    max_duration = max(max_duration, len(audio))  # Length is in milliseconds

# Initialize a silent audio segment with the maximum duration
final_audio = AudioSegment.silent(duration=max_duration)

# Loop through all audio files and overlay them on the final audio
for i, audio in enumerate(audio_segments):
    normalized_audio = normalize_audio(audio)
    # Apply the relative loudness adjustment based on the normalized loudness list
    adjusted_audio = normalized_audio + normalized_loudness_list[i]
    final_audio = final_audio.overlay(adjusted_audio)
    print(f"Overlayed audio {files[i]} - Duration: {final_audio.duration_seconds} seconds, Loudness Adjustment: {normalized_loudness_list[i]} dB")

# Export the final mixed audio
final_audio.export(os.path.join(save_dir, "final_output.wav"), format="wav")


Normalized loudness list: [8, 9, 10, 3, 7, 7, 0]
Overlayed audio 20250328234404.wav - Duration: 2.48 seconds, Loudness Adjustment: 8 dB
Overlayed audio 20250328235256.wav - Duration: 2.48 seconds, Loudness Adjustment: 9 dB
Overlayed audio 20250328235257.wav - Duration: 2.48 seconds, Loudness Adjustment: 10 dB
Overlayed audio 20250328234405.wav - Duration: 2.48 seconds, Loudness Adjustment: 3 dB
Overlayed audio 20250328235258.wav - Duration: 2.48 seconds, Loudness Adjustment: 7 dB
Overlayed audio 20250328234407.wav - Duration: 2.48 seconds, Loudness Adjustment: 7 dB
Overlayed audio 20250328234406.wav - Duration: 2.48 seconds, Loudness Adjustment: 0 dB


<_io.BufferedRandom name='example/results/Happy_Birthday_Jeff!/final_output.wav'>

Attempt to match features using DTW

In [44]:
import os
import librosa
import numpy as np
from pydub import AudioSegment
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Normalize audio loudness
def normalize_audio(audio, target_dBFS=-20.0):
    change_in_dBFS = target_dBFS - audio.dBFS
    return audio.apply_gain(change_in_dBFS)

# Calculate dynamic local speed ratios using DTW
def calculate_local_speed_ratios(reference, target, sr):
    ref_mel = librosa.feature.melspectrogram(y=reference, sr=sr, n_mels=128)
    tar_mel = librosa.feature.melspectrogram(y=target, sr=sr, n_mels=128)
    distance, path = fastdtw(ref_mel.T, tar_mel.T, dist=euclidean)
    path = np.array(path)

    # Calculate local speed ratios (slope between consecutive points)
    speed_ratios = np.ones(len(target))
    for i in range(1, len(path)):
        ref_step = path[i, 0] - path[i - 1, 0]
        tar_step = path[i, 1] - path[i - 1, 1]
        if tar_step > 0:
            local_speed = ref_step / (tar_step + 1e-6)
            speed_ratios[path[i, 1]] = local_speed
    print(speed_ratios)
    return speed_ratios


# Adjust local speed dynamically
def adjust_local_speed(audio, speed_ratios, sr):
    adjusted_audio = np.array([], dtype=np.float32)
    segment_length = 14096  # Frame length for adjustment

    for i in range(0, len(audio), segment_length):
        print("len audio",len(audio))
        segment = audio[i:i + segment_length]
        speed_ratio = np.mean(speed_ratios[i:i + segment_length])
        
        # Avoid zero or negative speed ratios
        if speed_ratio <= 0:
            speed_ratio = 1.0
        
        # Ensure the segment is long enough to time-stretch
        if len(segment) > 1:
            # Adjust the speed of the entire segment
            stretched_segment = librosa.effects.time_stretch(segment.astype(np.float32), rate=speed_ratio)
        else:
            stretched_segment = segment

        # Concatenate the adjusted segment to the output audio
        adjusted_audio = np.concatenate((adjusted_audio, stretched_segment))

    return adjusted_audio

# Directory containing the WAV files
files = [f for f in os.listdir(save_dir) if f.endswith('.wav')]

# Determine the maximum duration among all audio files
audio_segments = []
aligned_segments = []

# Load the first audio as the reference
reference_audio_path = os.path.join(save_dir, files[0])
reference_audio, sample_rate = librosa.load(reference_audio_path, sr=None)
audio_segments.append(reference_audio)

# Load and normalize other audios
for file in files[1:]:
    file_path = os.path.join(save_dir, file)
    audio, sr = librosa.load(file_path, sr=sample_rate)
    audio_segments.append(audio)

# Align all other audios to the reference by dynamically adjusting local speed
aligned_segments = []

for i, samples in enumerate(audio_segments):
    if i == 0:
        aligned_segments.append(samples)
        continue

    # Calculate local speed ratios using DTW
    speed_ratios = calculate_local_speed_ratios(reference_audio, samples, sample_rate)
    print(f"Audio {files[i]} - Calculated Local Speed Ratios: {speed_ratios[:10]}...")

    # Adjust local speed dynamically
    adjusted_samples = adjust_local_speed(samples, speed_ratios, sample_rate)
    aligned_segments.append(adjusted_samples)

# Convert aligned audio segments to pydub AudioSegment objects
audio_segments = [
    AudioSegment(
        (aligned * (2**15)).astype(np.int16).tobytes(),
        frame_rate=sample_rate,
        sample_width=2,
        channels=1
    ) for aligned in aligned_segments
]

# Find the maximum duration among aligned segments
max_duration = max(len(a) for a in audio_segments)

# Initialize the final audio segment with silence of the maximum duration
final_audio = AudioSegment.silent(duration=max_duration)

# Apply loudness normalization to each segment
loudness_list = [random.randint(1, 100) for _ in range(len(audio_segments))]

# Normalize loudness list to range of -10 to 10 dB
def normalize_loudness(loudness_list):
    max_value = max(abs(x) for x in loudness_list)
    if max_value == 0:
        return [0] * len(loudness_list)
    return [int((x / max_value) * 10) for x in loudness_list]

normalized_loudness_list = normalize_loudness(loudness_list)

# Overlay the aligned audios with loudness adjustment
for i, audio in enumerate(audio_segments):
    normalized_audio = normalize_audio(audio)
    adjusted_audio = normalized_audio + normalized_loudness_list[i]
    # Accumulate overlays by mixing with the final_audio
    final_audio = final_audio.overlay(adjusted_audio, position=0)
    print(f"Overlayed audio {files[i]} - Duration: {final_audio.duration_seconds} seconds, Loudness Adjustment: {normalized_loudness_list[i]} dB")

# Export the final mixed audio
final_output_path = os.path.join(save_dir, "final_output.wav")
final_audio.export(final_output_path, format="wav")
print(f"Final mixed audio saved to: {final_output_path}")


[1.       0.       0.999999 ... 1.       1.       1.      ]
Audio 20250328235256.wav - Calculated Local Speed Ratios: [1.       0.       0.999999 0.999999 0.999999 0.999999 0.999999 0.
 0.       0.999999]...
len audio 33600
len audio 33600
len audio 33600
[1.       0.999999 0.999999 ... 1.       1.       1.      ]
Audio final_output.wav - Calculated Local Speed Ratios: [1.       0.999999 0.999999 0.999999 0.       0.       0.       0.999999
 0.       0.999999]...
len audio 39680
len audio 39680
len audio 39680
[1.       0.999999 0.999999 ... 1.       1.       1.      ]
Audio 20250328235257.wav - Calculated Local Speed Ratios: [1.       0.999999 0.999999 0.999999 0.999999 0.999999 0.999999 0.
 0.       0.      ]...
len audio 19200
len audio 19200
[1.       0.999999 0.999999 ... 1.       1.       1.      ]
Audio 20250328234405.wav - Calculated Local Speed Ratios: [1.       0.999999 0.999999 0.999999 0.999999 0.999999 0.       0.
 0.       0.      ]...
len audio 16000
len audio 16000
[1. 

/home/qihan/miniconda3/envs/sparktts/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1904
  warnings.warn(
